# Pruebas modificaciones sobre datos 15/04/2024

In [49]:
import pandas as pd
import json
import shutil

# Mapeo de nombres de días de la semana de español a inglés
dias_esp_ing = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday'
}

# Paso 1: Copiar el archivo datosProcesados.csv a datosProcesados_modificado.csv
shutil.copyfile('datosProcesados.csv', 'datosProcesados_modificado.csv')

# Paso 2: Abrir el archivo datosProcesados_modificado.csv
df_modificado = pd.read_csv('datosProcesados_modificado.csv')

# Paso 3: Convertir la columna de fechas al formato utilizado en modificaciones.json
df_modificado['Fecha de la lectura'] = pd.to_datetime(df_modificado['Fecha de la lectura'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Paso 4: Abrir el archivo modificaciones.json
with open('modificaciones.json', 'r') as file:
    modificaciones = json.load(file)

# Paso 5: Iterar sobre cada modificación
for modificacion in modificaciones:
    opcion_seleccionada = modificacion.get('opcion_seleccionada')
    inicio_modificacion = modificacion.get('inicio')
    fin_modificacion = modificacion.get('fin')
    dias_modificacion_esp = modificacion.get('dias')  # Obtener la lista de días de la modificación en español
    porcentaje = modificacion.get('porcentaje') / 100  # Convertir porcentaje a decimal

    if opcion_seleccionada == 'aumentar_porcentaje' and inicio_modificacion and fin_modificacion and dias_modificacion_esp:
        # Convertir los nombres de los días de la semana de español a inglés
        dias_modificacion_ing = [dias_esp_ing[dia] for dia in dias_modificacion_esp if dia in dias_esp_ing]

        # Convertir inicio y fin a objetos datetime
        inicio_modificacion = pd.to_datetime(inicio_modificacion)
        fin_modificacion = pd.to_datetime(fin_modificacion)

        # Filtrar el DataFrame para obtener las filas dentro del rango de fechas de la modificación y en los días especificados
        df_filtrado = df_modificado[(df_modificado['Fecha de la lectura'] >= inicio_modificacion.strftime('%Y-%m-%d')) &
                                     (df_modificado['Fecha de la lectura'] <= fin_modificacion.strftime('%Y-%m-%d')) &
                                     (df_modificado['Dia de la semana'].isin(dias_modificacion_ing))]  # Filtrar por días traducidos

        print("DataFrame filtrado:")
        print(df_filtrado)

        print("Porcentaje de aumento:", porcentaje)

        # Convertir la columna 'Consumo (kwh)' a tipo float
        df_modificado['Consumo (kwh)'] = df_modificado['Consumo (kwh)'].astype(float)

        # Antes de aplicar el porcentaje de aumento
        print("Valores antes del aumento:")
        print(df_modificado.loc[df_filtrado.index, 'Consumo (kwh)'])

        # Aplicar el porcentaje de aumento al consumo en cada fila del DataFrame filtrado
        df_modificado.loc[df_filtrado.index, 'Consumo (kwh)'] *= (1 + porcentaje)

        # Después de aplicar el porcentaje de aumento
        print("Valores después del aumento:")
        print(df_modificado.loc[df_filtrado.index, 'Consumo (kwh)'])

        # Convertir la columna 'Consumo (kwh)' de nuevo a tipo int64
        df_modificado['Consumo (kwh)'] = df_modificado['Consumo (kwh)'].astype(int)

        print("Se han aplicado las modificaciones.")

# Paso 6: Guardar el DataFrame modificado en un nuevo archivo CSV
df_modificado.to_csv('datosProcesadosModificados.csv', index=False)
#df_modificado.to_excel('datosProcesadosModificados.xlsx', index=False)




DataFrame filtrado:
      Unnamed: 0 Cups                                       \
8732        8732                    ES0022000005731860JV1P   
8733        8733                    ES0022000005731860JV1P   
8734        8734                    ES0022000005731860JV1P   
8735        8735                    ES0022000005731860JV1P   
8736        8736                    ES0022000005731860JV1P   
...          ...                                       ...   
9879        9879                    ES0022000005731860JV1P   
9880        9880                    ES0022000005731860JV1P   
9881        9881                    ES0022000005731860JV1P   
9882        9882                    ES0022000005731860JV1P   
9883        9883                    ES0022000005731860JV1P   

     Fecha de la lectura Hora de la lectura  Consumo (kwh)  Hora inicio  \
8732          2018-04-02               0:15              8            0   
8733          2018-04-02               0:30              7            0   
8734      

In [37]:
df_modificado.loc[df_filtrado.index, 'Consumo (kwh)']

8732    16
8733    14
8734    16
8735    16
8736    14
        ..
9879    18
9880    14
9881    16
9882    16
9883    14
Name: Consumo (kwh), Length: 384, dtype: int64

In [41]:
df_modificado.loc[df_filtrado.index]


,Unnamed: 0,Cups,Fecha de la lectura,Hora de la lectura,Consumo (kwh),Hora inicio,Minutos inicio,Dia de la semana,Mes,Año,Periodo
8732,8732,ES0022000005731860JV1P,2018-04-02,0:15,16,0,0,Monday,4,2018,P6
8733,8733,ES0022000005731860JV1P,2018-04-02,0:30,14,0,15,Monday,4,2018,P6
8734,8734,ES0022000005731860JV1P,2018-04-02,0:45,16,0,30,Monday,4,2018,P6
8735,8735,ES0022000005731860JV1P,2018-04-02,1:00,16,0,45,Monday,4,2018,P6
8736,8736,ES0022000005731860JV1P,2018-04-02,1:15,14,1,0,Monday,4,2018,P6
...,...,...,...,...,...,...,...,...,...,...,...
9879,9879,ES0022000005731860JV1P,2018-04-13,23:00,18,22,45,Friday,4,2018,P5
9880,9880,ES0022000005731860JV1P,2018-04-13,23:15,14,23,0,Friday,4,2018,P5
9881,9881,ES0022000005731860JV1P,2018-04-13,23:30,16,23,15,Friday,4,2018,P5
9882,9882,ES0022000005731860JV1P,2018-04-13,23:45,16,23,30,Friday,4,2018,P5


In [42]:
df_modificado['Consumo (kwh)'] = pd.to_numeric(df_modificado['Consumo (kwh)'], errors='coerce')


In [43]:
print(df_modificado.loc[df_filtrado.index])


      Unnamed: 0 Cups                                       \
8732        8732                    ES0022000005731860JV1P   
8733        8733                    ES0022000005731860JV1P   
8734        8734                    ES0022000005731860JV1P   
8735        8735                    ES0022000005731860JV1P   
8736        8736                    ES0022000005731860JV1P   
...          ...                                       ...   
9879        9879                    ES0022000005731860JV1P   
9880        9880                    ES0022000005731860JV1P   
9881        9881                    ES0022000005731860JV1P   
9882        9882                    ES0022000005731860JV1P   
9883        9883                    ES0022000005731860JV1P   

     Fecha de la lectura Hora de la lectura  Consumo (kwh)  Hora inicio  \
8732          2018-04-02               0:15             16            0   
8733          2018-04-02               0:30             14            0   
8734          2018-04-02      

In [44]:
df_modificado.to_csv('datosProcesadosGuay.csv', index=False)


In [47]:
df_modificado.loc[df_filtrado.index]

,Unnamed: 0,Cups,Fecha de la lectura,Hora de la lectura,Consumo (kwh),Hora inicio,Minutos inicio,Dia de la semana,Mes,Año,Periodo
8732,8732,ES0022000005731860JV1P,2018-04-02,0:15,16,0,0,Monday,4,2018,P6
8733,8733,ES0022000005731860JV1P,2018-04-02,0:30,14,0,15,Monday,4,2018,P6
8734,8734,ES0022000005731860JV1P,2018-04-02,0:45,16,0,30,Monday,4,2018,P6
8735,8735,ES0022000005731860JV1P,2018-04-02,1:00,16,0,45,Monday,4,2018,P6
8736,8736,ES0022000005731860JV1P,2018-04-02,1:15,14,1,0,Monday,4,2018,P6
...,...,...,...,...,...,...,...,...,...,...,...
9879,9879,ES0022000005731860JV1P,2018-04-13,23:00,18,22,45,Friday,4,2018,P5
9880,9880,ES0022000005731860JV1P,2018-04-13,23:15,14,23,0,Friday,4,2018,P5
9881,9881,ES0022000005731860JV1P,2018-04-13,23:30,16,23,15,Friday,4,2018,P5
9882,9882,ES0022000005731860JV1P,2018-04-13,23:45,16,23,30,Friday,4,2018,P5


In [27]:
print(df_modificado.dtypes)


Unnamed: 0                                    int64
Cups                                         object
Fecha de la lectura                          object
Hora de la lectura                           object
Consumo (kwh)                                 int64
Hora inicio                                   int64
Minutos inicio                                int64
Dia de la semana                             object
Mes                                           int64
Año                                           int64
Periodo                                      object
dtype: object
